# EchoFlow 2.0 - Training on Kaggle
## World-Class Voice Pathology Detection

This notebook trains the EchoFlow 2.0 model using:
- **Wav2Vec2** for feature extraction
- **Transformer** for classification
- **Advanced augmentation** for robustness

### Expected Results:
- Accuracy: 92-95%
- Sensitivity: 90-94%
- Specificity: 93-96%

## 1. Setup Environment

In [ ]:
# Install dependencies
!pip install -q transformers librosa soundfile torchaudio

import torch
import numpy as np
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Download Dataset

Download Saarbruecken Voice Database from Zenodo.

In [ ]:
# Download dataset (17.9 GB)
!wget -c https://zenodo.org/records/16874898/files/data.zip -O /kaggle/working/data.zip

# Extract
!unzip -q /kaggle/working/data.zip -d /kaggle/working/dataset

# Check structure
!ls -lh /kaggle/working/dataset/

## 3. Clone EchoFlow 2.0 Code

In [ ]:
# Clone repository (replace with your repo URL)
!git clone https://github.com/izoomlentoboy-creator/Voice.git /kaggle/working/echoflow

# Navigate to project
%cd /kaggle/working/echoflow

## 4. Train Model

In [ ]:
# Train EchoFlow 2.0
!python train.py \
    --data_dir /kaggle/working/dataset \
    --batch_size 16 \
    --epochs 50 \
    --lr 1e-4 \
    --save_dir /kaggle/working/checkpoints \
    --num_workers 2

## 5. Evaluate Model

In [ ]:
# Load best model
import json

with open('/kaggle/working/checkpoints/history.json', 'r') as f:
    history = json.load(f)

print("Training History:")
print(f"Best Validation Accuracy: {max(history['val_acc']):.2f}%")
print(f"Final Train Accuracy: {history['train_acc'][-1]:.2f}%")
print(f"Final Val Accuracy: {history['val_acc'][-1]:.2f}%")

## 6. Save Model for Deployment

In [ ]:
# Package model for deployment
!cp /kaggle/working/checkpoints/best.pt /kaggle/working/echoflow_v2_best.pt

print("Model saved to: /kaggle/working/echoflow_v2_best.pt")
print("Download this file and deploy to your server!")

## 7. Test Inference

In [ ]:
# Test model on a sample
from models.transformer_classifier import EchoFlowV2

# Load model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = EchoFlowV2.load_model('/kaggle/working/echoflow_v2_best.pt', device=device)

# Test on a sample audio file
test_audio = '/kaggle/working/dataset/pathological/sample.wav'  # Replace with actual file
result = model.predict(test_audio)

print("Prediction Result:")
print(json.dumps(result, indent=2))